# Testing EUGENE utilities and PyTorch integrations

**Authorship:**
Adam Klie, *03/22/2022*
***
**Description:**
Notebook for testing out useful Pytorch functionality and EUGENE utilities. Kind of miscellaneous testing

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li>Optuna integration</li></b>
    <b><li></li></b>
    </ul>
</div>

In [1]:
# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import sys
sys.path.append("../scripts")

# Load a model and some test data

## Load model

In [17]:
from dsEUGENE import dsEUGENE

cnn=dict(input_len=66, channels=[4, 16], conv_kernels=[15, 5], pool_kernels=[1, 1])
rnn=dict(output_dim=32, batch_first=True)
fc=dict(output_dim=1)
eugene = dsEUGENE(conv_kwargs=cnn, rnn_kwargs=rnn, fc_kwargs=fc)

x = torch.randn(10, 4, 66)
out = eugene(x, x)
out.shape

## Load data

In [19]:
from torch.utils.data import dataset
from torch.utils.data import DataLoader

training_dataset = dataset.TensorDataset(torch.validationn(1000, 4, 66), torch.bernoulli(torch.empty(1000).uniform_(0,1)))
training_dataloader = DataLoader(training_dataset, batch_size=32, num_workers=4)

for i_batch, batch in enumerate(training_dataloader):
    x, y = batch
    outs = eugene(x)
    print(x.shape, y.shape, outs.shape)
    if i_batch==3:
        break

# DataModule

## Test random_split torch function

In [34]:
from torch.utils.data import random_split

In [38]:
dataset_len = len(training_dataset)
train_len = int(dataset_len*0.9)
val_len = dataset_len - train_len

In [40]:
train_dataset, val_dataset = random_split(training_dataset, [train_len, val_len])

In [42]:
len(train_dataset)

272642

# Torchmetrics logging

In [21]:
import torchmetrics

In [48]:
acc = torchmetrics.Accuracy()
preds = torch.round(torch.sigmoid(outs))
y.long().unsqueeze(dim=1)
acc.update(preds, y.long().unsqueeze(dim=1))
acc.compute()

In [91]:
auroc = torchmetrics.AUROC()
auroc.update(torch.sigmoid(outs), y.long().unsqueeze(dim=1))
auroc.compute()

# Optuna integration

In [2]:
from MPRADataModule import MPRADataModule

## Load data

In [3]:
from torchvision import transforms
from transforms import ReverseComplement, Augment, OneHotEncode, ToTensor

In [4]:
data_transform = 

In [6]:
OLS_TSV = "/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/2021_OLS_Library.tsv"
mod = MPRADataModule(seq_file=OLS_TSV,
                     transform=data_transform,
                     num_workers=4,
                     batch_size=128,
                     load_kwargs=dict(target_col="ACTIVITY_SUMRNA_NUMDNA", low_thresh=0.18, high_thresh=0.4))

Compose(
)


In [10]:
mod.setup()

/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:474: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  fn(*args, **kwargs)


Compose(
)


In [14]:
mod.train_dataloader().dataset[0]["sequence"].shape

torch.Size([4, 66])

## Test objective

In [13]:
from dsEUGENE import objective

In [14]:
import optuna

In [15]:
trial = optuna.trial.FixedTrial({"fcn_n_layers": 1, "fcn_dropout":0.2, "fcn_n_units_l0":4})

In [22]:
objective(trial, mod, max_epochs=2)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name         | Type                 | Params
------------------------------------------------------
0 | convnet      | BasicConv1D          | 976   
1 | recurrentnet | BasicRecurrent       | 8 K   
2 | fcnet        | FullyConnectedModule | 137   
3 | accuracy     | Accuracy             | 0     
4 | auroc        | AUROC                | 0     


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

0.7410488128662109

In [23]:
trial.number

0

In [8]:
study = optuna.create_study(direction="maximize")

[I 2022-03-21 19:31:52,680] A new study created in memory with name: no-name-33c62b8b-30b6-4731-afa9-5017148cd173


In [16]:
from optuna.distributions import UniformDistribution

In [17]:
trial = optuna.trial.create_trial(
    params={"x": 2.0},
    distributions={"x": UniformDistribution(0, 10)},
    value=4.0,
)

/cellar/users/aklie/opt/miniconda3/envs/seq_tools_dev/lib/python3.7/site-packages/ipykernel_launcher.py:4: ExperimentalWarning: create_trial is experimental (supported from v2.0.0). The interface can change in the future.
  after removing the cwd from sys.path.


In [18]:
study.add_trial(trial)

/cellar/users/aklie/opt/miniconda3/envs/seq_tools_dev/lib/python3.7/site-packages/ipykernel_launcher.py:1: ExperimentalWarning: add_trial is experimental (supported from v2.0.0). The interface can change in the future.
  """Entry point for launching an IPython kernel.


In [19]:
objective(trial)

ValueError: The value of the parameter 'fcn_n_layers' is not found. Please set it at the construction of the FrozenTrial object.

# Config Generation

In [43]:
import inspect
import questionary
import importlib
from eugene.models.hybrid import hybrid
from eugene.models.cnn import CNN
from eugene.models.fcn import FCN
from eugene.models.rnn import RNN
from claim.modules._base_modules import BasicFullyConnectedModule, BasicConv1D, BasicRecurrent

In [ ]:
getattr(importlib.import_module(f"eugene.models.

In [47]:
model_type = "hybrid"
if model_type in ["fcn", "cnn", "rnn"]:
        module = getattr(importlib.import_module(f"eugene.models.{model_type}"), model_type.upper())
elif model_type in ["hybrid"]:
    module = getattr(importlib.import_module(f"eugene.models.{model_type.lower()}"), model_type.lower())

In [62]:
init_params = inspect.signature(module.__init__).parameters

In [67]:
init_params["input_len"].default

inspect._empty

In [59]:
higher_params = []
for key in init_params.keys():
    param_name = init_params[key].name
    param_default = init_params[key].default if type(init_params[key].default) == str else ""
    if param_name == "self":
        continue
    print(f"{param_name} (default: {param_default})")
    if param_name in ["fc_kwargs", "conv_kwargs", "rnn_kwargs"]:
        if param_name == "fc_kwargs":
            module_params = inspect.signature(BasicFullyConnectedModule.__init__).parameters
        elif param_name == "conv_kwargs":
            module_params = inspect.signature(BasicConv1D.__init__).parameters
        elif param_name == "rnn_kwargs":
            module_params = inspect.signature(BasicRecurrent.__init__).parameters
        for module_key in module_params:
            module_name = module_params[module_key].name
            module_default = module_params[module_key].default if type(module_params[module_key].default) == str else ""
            if module_name == "self":
                continue
            if module_name in higher_params:
                continue
            print(f"\t{module_name} (default: {module_default})")
    higher_params.append(param_name)

#questionary.text(f"{param_name} (default: {param_default})").ask()

input_len (default: )
strand (default: ss)
task (default: regression)
aggr (default: )
conv_kwargs (default: )
	channels (default: )
	conv_kernels (default: )
	pool_kernels (default: )
	activation (default: relu)
	pool_strides (default: )
	dropout_rates (default: )
	batchnorm (default: )
rnn_kwargs (default: )
	input_dim (default: )
	output_dim (default: )
	unit_type (default: lstm)
	bidirectional (default: )
	kwargs (default: )
fc_kwargs (default: )
	input_dim (default: )
	output_dim (default: )
	hidden_dims (default: )
	kwargs (default: )


In [61]:
questionary.text(f"{param_name} (default: {param_default})").ask()

RuntimeError: This event loop is already running

In [70]:
import ast

In [82]:
test = 'test'

In [85]:
ast.literal_eval(f"{test}")

ValueError: malformed node or string: <_ast.Name object at 0x1554ba8e8610>

In [77]:
reveal_type

NameError: name 'reveal_type' is not defined

In [146]:
def infer_type(user_input):
    if "[" in user_input:
        try:
            return [int(i) for i in user_input.strip('][').split(', ')] 
        except:
            pass
    if "." in user_input:
        try:
            return float(user_input)
        except:
            pass
    else:
        try:
            return int(user_input)
        except:
            pass
    return user_input

In [148]:
tests = ['[1, 2]', '1.0', '1', 'string']
[infer_type(test) for test in tests]

[[1, 2], 1.0, 1, 'string']

[1, 2]

In [86]:

    val = ast.literal_eval(inp)
    acceptable_types = (int, float)
    if any(isinstance(val, x) for x in acceptable_types):
        return val
    raise SomeException("Bad Input")

In [87]:
infer_type("string")

ValueError: malformed node or string: <_ast.Name object at 0x1554ba8ef150>